# Ibis for dplyr Users

[R](https://www.r-project.org/) users familiar with [dplyr](https://dplyr.tidyverse.org/) and other packages in the [Tidyverse](https://www.tidyverse.org/) are likely find Ibis familiar.
In fact, some Ibis features were even inspired by similar features in the [Tidyverse](https://www.tidyverse.org/).

However, due to differences between Python and R and the design and goals of Ibis itself, you may notice some big differences right away:

- **No pipe:** The handy [magrittr pipe](https://magrittr.tidyverse.org/) (`%>%`) or R's newer native pipe (`|>`) don't exist in Python so you instead have to chain sequences of operations together with a period (`.`).
- **No unquoted column names:** Non-standard evaluation is common in R but not present in Python. To reference a column in Ibis, you can pass a string, property on a table (e.g., `tbl.some_column`), or you can make use of [selectors](https://ibis-project.org/api/selectors/).
- **Ibis is lazy by default:** Similar to [dbplyr](https://dbplyr.tidyverse.org/) and its `collect()` method, Ibis does not evaluate our queries until we call `.execute()`. For the purposes of this document, we set `ibis.options.interactive = True` which calls `execute` automatically for us and prints a nicely-formatted table.

## Comparing Ibis and dplyr

Using the same example data and similar operations as in [Introduction to dplyr](https://dplyr.tidyverse.org/articles/dplyr.html), below you will find some examples of the more common dplyr operations and their Ibis equivalents.

### Loading Ibis

To start using dplyr in R we would run:

```r
library(dplyr)
```

When loading Ibis, it's useful to first load Ibis:

In [ ]:
import ibis

And then also load and alias some helpers to make our code more concise:

In [ ]:
import ibis.selectors as s
from ibis import _

Last, as mentioned above, to get Ibis to automatically execute our queries and show the results in a nicely-formatted table, we run:

In [ ]:
ibis.options.interactive = True

### Loading example data

In R, datasets are typically lazily loaded with packages. For instance, the `starwars` dataset is packaged with dplyr, but is not loaded in memory before you start using it. Ibis provides many datasets in the `examples` module. So to be able to use the `starwars` dataset, you can use:

In [ ]:
starwars = ibis.examples.starwars.fetch()

Similar to dplyr, if we evaluate the name of a table, we get a nicely-formatted table:

In [ ]:
starwars

### Inspecting the dataset with `head()`

Just like in R, you can use `head()` to inspect the beginning of a dataset. You can also specify the number of rows you want to get back by using the parameter `n` (default `n = 5`).

In R:

```r
head(starwars) # or starwars |> head()
```

With Ibis:

In [ ]:
starwars.head(6)

There is no `tail()` in Ibis because most databases do not support this operation.

Another method you can use to limit the number of rows returned by a query is `limit()` which also takes the `n` parameter.

In [ ]:
starwars.limit(3)

### Filtering rows with filter()

Ibis, like dplyr, has the `filter` method to select rows based on conditions.

With dplyr:

```r
starwars |>
  filter(skin_color == "light")
```

In Ibis:

In [ ]:
starwars.filter(_.skin_color == "light")

In dplyr, you can specify multiple conditions separated with `,` that are then combined with the `&` operator:

```r
starwars |>
  filter(skin_color == "light", eye_color == "brown")
```

In Ibis, you can do the same by putting multiple conditions in a list:

In [ ]:
starwars.filter([_.skin_color == "light", _.eye_color == "brown"])

In previous code, we used the `_` helper we imported earlier. The `_` is shorthand for the table we referenced at the beginning of the expression (in this case, `starwars`). We could have also written the more verbose form,

```python
starwars.filter([starwars.skin_color == "light", starwars.eye_color == "brown"])
```

If you want to combine multiple conditions, in dplyr, you could do:

```r
starwars |>
  filter(
      (skin_color == "light" & eye_color == "brown") |
       species == "Droid"
  )
```

In Ibis, this would be:

In [ ]:
starwars.filter(
    ((_.skin_color == "light") & (_.eye_color == "brown")) |
    (_.species == "Droid")
)

### Sorting your data with order_by()

To sort a column, dplyr has the verb `arrange`. For instance, to sort the column `height` using dplyr:

```r
starwars |>
   arrange(height)
```

In Ibis:

In [ ]:
starwars.order_by(_.height)

You might notice that while dplyr puts missing values at the end, Ibis places them at the top.

If you want to order using multiple variables, you can pass them as a list:

In [ ]:
starwars.order_by([_.height, _.mass]) # or starwars.order_by(["heigh", "mass"])

To order a column in descending order, there are two ways to do it. Note that missing values remain at the top.

In [ ]:
starwars.order_by(_.height.desc()) # or: starwars.order_by(ibis.desc("height"))

### Selecting columns with select()

Ibis, like dplyr, has a `select` method to include or exclude columns:

With dplyr:

```r
starwars |> 
    select(hair_color)
```

In Ibis:

In [ ]:
starwars.select(_.hair_color)

dplyr also allows selecting more than one column at a time:

```r
starwars |>
    select(hair_color, skin_color, eye_color)
```

In Ibis, you can pass a list of column names:

In [ ]:
starwars.select(["hair_color", "skin_color", "eye_color"])

To select columns by name based on a condition, dplyr has helpers such as:

- starts_with(): Starts with a prefix.
- ends_with(): Ends with a suffix.
- contains(): Contains a literal string.

These and many more [selectors](https://ibis-project.org/api/selectors/) are available in Ibis as well, with slightly different names:

In [ ]:
starwars.select(s.startswith("h"))

In [ ]:
starwars.select(s.endswith("color"))

In [ ]:
starwars.select(s.contains("world"))

See the Ibis [Column Selectors](https://ibis-project.org/api/selectors/) documentation for the full list of selectors in Ibis.

### Renaming columns with relabel()

Ibis allows you to rename columns using `relabel()` which provides similar functionality to `rename()` in dplyr.

In dplyr:

```r
starwars |> 
    rename("homeworld" = "home_world")
```

In Ibis, use `relabel` and pass a `dict` or name mappings:

In [ ]:
starwars.relabel({"homeworld": "home_world"})

### Add new columns with mutate()

Ibis, like dplyr, uses the `mutate` verb to add columns.

In dplyr,

```r
starwars |>
    mutate(height_m = height / 100) |>
    select(name, height_m)
```

In Ibis:

In [ ]:
(
    starwars
        .mutate(height_m = _.height / 100)
        .select("name", "height_m")
)

A big difference between dplyr's `mutate` and Ibis' is that, in Ibis, you have to chain separate `mutate` calls together you reference newly-created columns in the same `mutate` whereas in dplyr, you can put them all in the same call. This makes Ibis' `mutate` more similar to `transform` in dplyr.

In dplyr, we only need one `mutate` call:

```r
starwars %>%
  mutate(
    height_m = height / 100,
    BMI = mass / (height_m^2)
  ) %>%
  select(BMI, everything())
  ```

In Ibis, for `BMI` to reference `height_m`, it needs to be in a separate `mutate` call:

In [ ]:
(starwars
    .mutate(
        height_m = _.height / 100
    )
    .mutate(        
        BMI = _.mass / (_.height_m**2)
    )
    .select("BMI", ~s.matches("BMI"))
)

### Summarize values with aggregate()

To summarize tables, dplyr has the verbs `summarize`/`summarize`:

In dplyr:

```r
starwars %>% 
    summarise(height = mean(height, na.rm = TRUE))
```

In Ibis, the corresponding verb is `aggregate`:

In [ ]:
starwars.aggregate(height = _.height.mean())

### Joins

To demonstrate how to do joins with Ibis, we'll load two more example datasets that also come from the example datasets included in dplyr:

In [ ]:
band_members = ibis.examples.band_members.fetch(header=1)
band_instruments = ibis.examples.band_instruments.fetch(header=1)

In dplyr, we can perform a left join of these two tables like:

```r
band_members |> 
    left_join(band_instruments)
```

This produces the output of:

```
Joining with `by = join_by(name)`
# A tibble: 3 × 4
  name.x band    name.y plays 
  <chr>  <chr>   <chr>  <chr> 
1 Mick   Stones  NA     NA    
2 John   Beatles John   guitar
3 Paul   Beatles Paul   bass  
```

In Ibis:

In [ ]:
band_members.left_join(
    band_instruments, 
    band_members.name == band_instruments.name
)

There are two main differences between Ibis and dplr here:

- Ibis requires us to explicitly specify our join key ("name", in this example) whereas dplyr will try to guess the join key for us
- Ibis keeps the join keys from each table whereas dplyr does not by default

Two replicate the result we'd get by default in dplyr but using Ibis, we need to incorporate two other verbs we've already seen in this tutorial:

In [ ]:
(
    band_members
        .left_join(
            band_instruments, 
            band_members.name == band_instruments.name)
        .select(~s.contains("_y"))
        .relabel({"name_x": "name"})
)

### Pivot wider/longer

dplyr users are likely to be familiar with the `pivot_wider` and `pivot_longer` functions from the [tidyr](https://tidyr.tidyverse.org) package which convert tables between wide and long formats, respectively.

In dplyr+tidyr:

```r
starwars |> 
    select(name, matches("color")) |> 
    pivot_longer(matches("color"), names_to = "attribute", values_to = "color")
```

In Ibis:

In [ ]:
starwars_colors = (
    starwars
        .select("name", s.matches("color"))
        .pivot_longer(s.matches("color"), names_to="attribute", values_to="color")
)

starwars_colors


And the reverse, in dplyr:

```r
starwars_colors |> 
    pivot_wider(names_from = "attribute", values_from = "value")
```

In Ibis:

In [ ]:
(
    starwars_colors.
        pivot_wider(names_from="attribute", values_from="color")
)

## Next Steps

Now that you've gotten an introudction to the common differences between dplyr and Ibis, head over to [Getting Started with ibis](https://ibis-project.org/getting_started/) for a full introduction. If you're familiar with SQL, check out [Ibis for SQL Programmers](https://ibis-project.org/ibis-for-sql-programmers/). If you're familiar with pandas, take a look at [Ibis for Pandas Users](https://ibis-project.org/ibis-for-pandas-users/.)